# References
[1] sckit-learn developers, 2017. [online] http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html. [Accessed: April 17, 2018]

[2] abhijeet3922, “Image-compression-with-Kmeans-clustering”, GitHub, 2017. [online] https://github.com/abhijeet3922/Image-compression-with-Kmeans-clustering/blob/master/img_compress.py. [Accessed: April 17, 2018] 

[3] Robert Layton, Olivier Grisel, Mathieu Blondel, “Color Quantization using K-Means”, scikit learn, 2017. [online] http://scikit-learn.org/stable/auto_examples/cluster/plot_color_quantization.html. [Accessed: April 17, 2018]

# How to Run the Code
1. Makes sure that the Image_Resize_Compressor.ipynb is placed in the same folder with all the image files. Ensure that the only folders in the same level as this code are the image files.
2. Run Each segment in the chronological order
    - Change the 
3. Out put will be folder named the same, but starting with 'Resize_Compress_'

# Inputs

In [ ]:
# Change the number of clusters as needed
n_colors = 10                                                # Set number of clusters

# Libraries

In [ ]:
# 542 Project
# Joseph O'Donnell
# Gautham Ravichandran
# Albert Sze
# Eric Zhang

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from sklearn.datasets import load_sample_image
from sklearn.utils import shuffle
from time import time
from PIL import Image
import scipy.misc
import pandas as pd
import random
import os, sys
import os.path
import glob
import math
home = os.getcwd()                                       # Get path name for home folder/current location

# Recreate Image

In [ ]:
def recreate_image(codebook, labels, h, w):
    """Recreate the (compressed) image from the code book & labels"""
    d = codebook.shape[1]                              # Finds the depths of the image
    image = np.zeros((h, w, d))                        # Creates a temporary image, full of zeros, of the same size of the original image
    label_idx = 0                                      # Starts at first pixel
    for i in range(h):                                 # Starts at the top of the image
        for j in range(w):                             # Starts at the left of the image
            image[i][j] = codebook[labels[label_idx]]  # Set pixel at (i,j) to respective centroid value/color
            label_idx += 1                             # Iterates to next pixel
    return image                                       # Returns reconstructed image

# Creating New Directories

In [ ]:
directory_list = []                                       # Creates an empty directory list
for root, dirs, files in os.walk(".", topdown=False):     # Iterates through all folder names
    for name in dirs:                                     
        directory_list.append(os.path.join(root, name))   # Adds folder names to directory list
j = 0
for i in range(len(directory_list)):                      # Iterate through directory list names
    if (directory_list[i-j][2:5] != '001'):               # Remove folders that don't have numbers in the front
        directory_list.remove(directory_list[i-j])
        j += 1
    else:
        break
j = 0        
for i in range(len(directory_list)):                      # Iterate through directory list names 
    if (directory_list[i-j][2:3] == 'R'):                 # Remove folders that start with 'R' 
        directory_list.remove(directory_list[i-j])
        j += 1

for i in directory_list:                                  # Iterate through directory list names
    fname = 'Resize_Compress_' + i[2:]                    
    if not os.path.isdir(i[:2] + fname):                  # Generate new folders with Resized_compress_ + original folder name if folder does not exist
        os.makedirs(fname)

# Resizing and Compressing Images

In [ ]:
t0 = time()
print('Resizing and Compressing Images')
for i in directory_list:                                     # Iterate through folders
    fname_rc = home + '\\' + 'Resize_Compress_' + i[2:]      # Name for path to new Resized and Compressed folder
    fname = home + '\\' + i[2:]                              # Name for path to original foldwer name
    os.chdir(i)                                              # Change directory
    for file in glob.glob("*.jpg"):                          # Iterate through all JPEG files
        image = Image.open(file)                             # Open JPEG image file
        imarray=np.array(image)                              # Convert Image to an array
        height = len(imarray)                                # Get the original height of the image
        width = len(imarray[0])                              # Get the original width of the image

        if height == width:                                  # If the height and width are equal to each other resize to 80 X 80
            size = 80, 80
        elif height < width:                                 # If the height is smaller than the width resize to height to 80 and width to ratio * 80
            size = int(math.ceil((width/height)*80)), 80
        else:                                                # If the height is larger than the width resize to height to ratio * 80 and width to 80
            size = 80, int(math.ceil((height/width)*80))     

        if (height < 80) | (width < 80):                     # If the original height or width is less than 80 resize using image.resize
            image = image.resize( ( size ) ) 
        else:                                                # Else use image.thumbnail
            image.thumbnail(size, Image.ANTIALIAS)
        imarray=np.array(image)                              # Convert new resized image to an array
        
        if imarray.ndim > 2:
            # Rescale values from 0-1 by dividing by 255. This is done so that plt.imshow works well
            image = np.array(image, dtype=np.float64) / 255

            # Load Image and transform to a 2D numpy array.
            h, w, d = original_shape = tuple(image.shape)
            image_array = np.reshape(image, (h * w, d))

            image_array_sample = shuffle(image_array, random_state=0)[:1000]                  # Selects 1000 random samples from the resized image
            # Trains K-mean algorithm with sample data with default of kmean++ (helps with initial cluster selection)
            kmeans = KMeans(n_clusters=n_colors, random_state=0).fit(image_array_sample)
            labels = kmeans.predict(image_array)                                              # Predicts cluster classification on entire resized image
        
        os.chdir(fname_rc)                                                                                      # Change directory to new Resized and Compressed folder
        if imarray.ndim <= 2:
            scipy.misc.imsave('resize_comp_' + file, imarray)                                                   # If image is original a grayscale image, just save resized image
        else:
            scipy.misc.imsave('resize_comp_' + file, recreate_image(kmeans.cluster_centers_, labels, h, w))     # If image is in color save resized and compressed image
        os.chdir(fname)                                                                                         # Change directory back to original image folder
    os.chdir(home)  # Change directory to home folder
os.chdir(home)  # Change directory to home folder
print("Done in %0.3fs." % (time() - t0))